![Banner](Recursos\Banner.png)

### Instalaremos paquetes y librerías necesarias para descargar, trabajar los DataSets para cumplir con nuestro MVP descrito en nuestro README.md

In [ ]:
# Importo librerías y paquetes necesarios
import numpy
import pandas as pd
import zipfile
import os
import json
import requests
import ssl
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from requests.adapters import HTTPAdapter
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from requests.packages.urllib3.poolmanager import PoolManager


### Trato advertencias

In [ ]:
# Deshabilito advertencias de solicitudes inseguras.
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

### Llamaré a los archivos con los que trabajaremos.

In [ ]:
# Rutas a los archivos ZIP y el endpoint de la API
credit_card_zip_path = r"C:\\Users\Esteban García\\OneDrive\\Escritorio\\LABs\\Datasets para Trabajar NoCountry\\DataFrames\\Credit Card Transactions Fraud Detection Dataset.zip"
ecommerce_zip_path = r"C:\\Users\\Esteban García\\OneDrive\\Escritorio\\LABs\\Datasets para Trabajar NoCountry\\DataFrames\\Fraudulent E-commerce Transactions.zip"

# Endpoint de la API del BCRA (ajustar según la documentación)
bcra_api_url = "https://api.bcra.gob.ar/estadisticas/v2.0/PrincipalesVariables"

### Establezco la ruta donde guardaré los df y *.csv

In [ ]:
output_dir = r"C:\\Users\\Esteban García\\OneDrive\\Escritorio\\LABs\\Datasets para Trabajar NoCountry\\DataFrames\\DF Procesados"

### Defino la función para explorar los documentos

In [ ]:
# Función para explorar y analizar los archivos *.zip

def list_files_in_zip(zip_path):
    """Listo los archivos dentro de un archivo ZIP."""
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        return zip_ref.namelist()


def extract_and_read_zip(zip_path, file_name):
    """Extraigo los archivos ZIP y leo los datasets."""
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall("temp_dataset")
        file_path = os.path.join("temp_dataset", file_name)
        return pd.read_csv(file_path)

### Verifico que la extracción de los archivos fue existosa.

In [ ]:
# Verifico los nombres de los archivos dentro de los ZIPs
credit_card_files = list_files_in_zip(credit_card_zip_path)
ecommerce_files = list_files_in_zip(ecommerce_zip_path)

# Verifico la lectura correcta de los archivos
credit_card_file_name = next((file for file in credit_card_files if file.endswith('.csv')), None)
ecommerce_file_name = next((file for file in ecommerce_files if file.endswith('.csv')), None)

if credit_card_file_name:
    credit_card_df = extract_and_read_zip(credit_card_zip_path, credit_card_file_name)
else:
    print("No se encontró un archivo CSV en el ZIP de tarjetas de crédito.")

if ecommerce_file_name:
    ecommerce_df = extract_and_read_zip(ecommerce_zip_path, ecommerce_file_name)
else:
    print("No se encontró un archivo CSV en el ZIP de comercio electrónico.")

### Consulto la APIs del BCRA

In [ ]:
# Determino la URL base de la API
base_url = "https://api.bcra.gob.ar/estadisticas/v2.0/"

# Establezco el endpoint para obtener principales variables
variables_url = base_url + "principalesvariables"

# Solicito los datos de principales variables
response = requests.get(variables_url, verify=False)
if response.status_code == 200:
    variables = response.json()['results']
else:
    print("Error al obtener principales variables:", response.status_code)

# Creo un DataFrame con las principales variables
bcra_df = pd.DataFrame(variables)

# Muestro las primeras filas del DataFrame
print(bcra_df.head())

# Guardar el DataFrame en un archivo CSV
output_dir = r"C:\\Users\\Esteban García\\OneDrive\\Escritorio\\LABs\\Datasets para Trabajar NoCountry\\DataFrames\\DF Procesados"
bcra_df.to_csv(output_dir + "principales_variables.csv", index=False)

# Función para obtener datos de una variable específica
def obtener_datos_variable(id_variable, desde, hasta):
    datos_url = f"{base_url}datosvariable/{id_variable}/{desde}/{hasta}"
    response = requests.get(datos_url, verify=False)
    if response.status_code == 200:
        return response.json()['results']
    else:
        print("Error al obtener datos de la variable:", response.status_code)
        return []

# Establezco el rango de fechas para el año 2024
id_variable = 1  # ID de la variable a consultar
desde = "2024-01-01"  # Fecha de inicio del rango
hasta = "2024-12-31"  # Fecha de fin del rango

datos_variable = obtener_datos_variable(id_variable, desde, hasta)

# Creo el DataFrame con los datos de la variable
bcra_df = pd.DataFrame(datos_variable)

# Mostrar las primeras filas del DataFrame de datos de la variable
print(bcra_df.head())

# Guardar el DataFrame en un archivo CSV
bcra_df.to_csv(output_dir + "datos_variable.csv", index=False)



### Visualizo los df para ver su contenido para tener un primer acercamiento al contenido, visualizarlo y entender los pasos a seguir.

In [ ]:
ecommerce_df

In [ ]:
credit_card_df

In [ ]:
bcra_df

### Al haber una reducción de personal a última hora, elegimos trabajar con el DataFrame llamado ecommerce_df

In [ ]:
ecommerce_df

### Modificaré la columna 'Transaction Date' en dos columnas una que contenga la fecha y la hora que contenga la hora, ambas en fomato datetime.

In [ ]:
ecommerce_df['Transaction Date'] = pd.to_datetime(ecommerce_df['Transaction Date'])
ecommerce_df['Transaction Time'] = ecommerce_df['Transaction Date'].dt.strftime('%H:%M:%S')
ecommerce_df['Transaction Date'] = ecommerce_df['Transaction Date'].dt.strftime('%Y-%m-%d')


print(ecommerce_df[['Transaction Date', 'Transaction Time']].head())


In [ ]:
ecommerce_df

### Análisis exploratorio de fraudes en e-commerce_df

In [ ]:
# Load the CSV file
file_path = 'ruta/a/tu/archivo/ecommerce.csv'
df = pd.read_csv(file_path)

# Convert 'Transaction Date' to datetime
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])

# Filter the dataset to include only fraudulent transactions
fraud_df = df[df['Is Fraudulent'] == 1]

# Check if there's a 'Gender' column, if not we'll use another categorical column
if 'Gender' in fraud_df.columns:
    gender_column = 'Gender'
else:
    gender_column = 'Payment Method'  # Using 'Payment Method' as an example

# Create subplots
fig, axs = plt.subplots(2, 2, figsize=(15, 12))

# 1. Distribution of frauds by gender (or payment method)
sns.countplot(x=gender_column, data=fraud_df, ax=axs[0, 0], palette="viridis")
axs[0, 0].set_title('Distribution of Frauds by ' + gender_column)
axs[0, 0].set_xlabel(gender_column)
axs[0, 0].set_ylabel('Count')

# 2. Distribution of frauds by transaction amount
sns.histplot(fraud_df['Transaction Amount'], kde=True, ax=axs[0, 1], color='blue')
axs[0, 1].set_title('Distribution of Fraudulent Transaction Amounts')
axs[0, 1].set_xlabel('Transaction Amount')
axs[0, 1].set_ylabel('Frequency')

# 3. Distribution of frauds over time
fraud_df.set_index('Transaction Date')['Transaction Amount'].resample('D').sum().plot(ax=axs[1, 0])
axs[1, 0].set_title('Fraudulent Transactions Over Time')
axs[1, 0].set_xlabel('Date')
axs[1, 0].set_ylabel('Total Fraudulent Amount')

# 4. Distribution of frauds by device used
sns.countplot(x='Device Used', data=fraud_df, ax=axs[1, 1], palette="viridis")
axs[1, 1].set_title('Distribution of Frauds by Device Used')
axs[1, 1].set_xlabel('Device Used')
axs[1, 1].set_ylabel('Count')

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd

# Suponiendo que ya tienes cargado tu DataFrame llamado ecommerce_df

# Obtener una muestra del 10% del DataFrame
sample_df = ecommerce_df.sample(frac=0.1)

# Especificar la ruta y nombre del archivo donde se guardará la muestra
file_path = "C:\\Users\\Esteban García\\OneDrive\\Escritorio\\LABs\\No Country\\ecommerce.csv"

# Guardar la muestra en un archivo CSV en la carpeta específica
sample_df.to_csv(file_path, index=False)

print("Muestra del 10% guardada exitosamente en:", file_path)